# ミニバッチ学習の実装

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math

# Load the MNIST dataset
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

train = X_train/255
test = X_test/255
train = train.reshape(-1, 28*28)
test = test.reshape(-1, 28*28)
train_labels = lb.fit_transform(y_train)
test_labels = lb.fit_transform(y_test)

2022-09-07 04:31:33.361189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-07 04:31:33.543886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 04:31:33.543987: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-07 04:31:33.617846: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-07 04:31:34.375333: W tensorflow/stream_executor/platform/de

In [14]:
# 255で割る理由＝正規化（最小：0、最大：1への変換）
# X_trainは60000個の画像データであり、各画像においては784（=28*28）個のセルの中に0〜255の数字が入っている
print(X_train.shape)
print(X_train.min(), X_train.max())

(60000, 28, 28)
0 255


## ミニバッチ学習
* ミニバッチ学習は、一般的には非復元抽出によって行われることが多いが、必ずこうしなければならないというわけではなく、分析者がデータセットの与え方を工夫することもできる。ただし、工夫しても計算が上手くいくとは限らない。
* 工夫のしどころ。
    * 一般的には、エポックのたびにシャッフルするが、シャッフルするタイミングを任意に変えてみる  
    * 与えるミニバッチ の順番を意図的に操作してみる   
        * 例、出現頻度の少ないラベルのデータを先に学習させる
    * 抽出されるラベルの割合が一定になるように抽出してみる
    * 復元抽出にしてみる

In [2]:
def trainer(network, x, y):
    """
    学習用の関数
    このnotebookでは、ミニバッチ学習を学ぶことが目的であるため、この関数の中身は空のままにしておく
    実際には、何らかの学習させるための処理を記述する
    """
    pass
    return 

### ミニバッチ学習のループ(復元抽出)

In [10]:
np.random.seed(1234)
train_size = train_labels.shape[0]
batch_size = 32
max_iter = 10  #ループの回数
network = None #ダミー

for i in range(max_iter):
    # 復元抽出 train_size Combination batch_sizeのイメージ
    batch_mask = np.random.choice(train_size, batch_size) 
    # バッチサイズは32としているが、ここではスペース省略のため10個のみ表示
    print("i=%s, "%i, "batch_mask=%s"%batch_mask[:10])
    # numpyではtrain[batch_mask]とすることで、batch_maskの要素番目のものを取り出してくることができる
    x_batch = train[batch_mask]
    y_batch = train_labels[batch_mask]

    # ここでは学習はしないので、ダミーのNetworkを与えている
    trainer(network, x_batch, y_batch)

i=0,  batch_mask=[27439 58067 34086 56373 23924 17048 55289 32399 55985 41239]
i=1,  batch_mask=[22267 21580 14629 12198 50682 46275 10983 23691 39552 21225]
i=2,  batch_mask=[29280 27532 28869 31741 49777  3039  8165 28319  8953 29692]
i=3,  batch_mask=[50926 18991  3062 43252 43382 58031 58255 51746    10  3356]
i=4,  batch_mask=[48917 55492 14455 34791 59642 43444 43456 20949  4468 43881]
i=5,  batch_mask=[12184 57310 45801 55823  6984  4994 52624   619 21634 19186]
i=6,  batch_mask=[27117 42059  4111 34580  4880 36288 26464 32382 26835 40249]
i=7,  batch_mask=[ 9624 10252 10700 18272  7688 13615 12057 51949 55061 35990]
i=8,  batch_mask=[49068 54819 35754 49556 43802 12633 59499 36759 32386 47848]
i=9,  batch_mask=[ 9532 31315 31088 29429 21129 37436 32946 35249 59498 17095]


### 復元抽出部分を理解するためのコード

In [11]:
np.random.seed(1234)
batch_mask = np.random.choice(train_size, batch_size)
# 32件のデータを取り出す
print("batch_mask=", batch_mask)
print()

x_batch = train[batch_mask]
print("x_batch=", x_batch)
# ３２件のデータを取り出す。それぞれ784個の要素で構成されるデータである
print("x_batch.shape=", x_batch.shape)
print()
y_batch = train_labels[batch_mask]
print("y_batch=", y_batch)
# 32件のデータを取り出す。それぞれ10個の要素で構成されるデータである
# ここでは0, 1, ..., 9のどの数字を指しているかを示す
print("y_batch.shape=", y_batch.shape)
print()

batch_mask= [27439 58067 34086 56373 23924 17048 55289 32399 55985 41239  9449 23706
  8222 32427 33950 40684  8060  7962 13686 59834 59512 14192  7644 27973
 27984 41929 51583 49398  2558 36271 38450  3824]

x_batch= [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
x_batch.shape= (32, 784)

y_batch= [[0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 

In [7]:
# 復元抽出部分(何回か実行してみてください)
np.random.choice(10,3)

array([9, 0, 3])

### ミニバッチ学習のループ(非復元抽出)

### [演習]
* 以下の非復元抽出によるミニバッチ学習を完成させましょう。
* 通常の計算では、非復元抽出で行うことが多いです。

In [8]:
# ヒント
index = np.arange(10)
print("index=%s"%index)
np.random.seed(1234)
np.random.shuffle(index)
print("index=%s"%index)
print()
print(np.random.permutation(10))
print()

for i in range(4):
    print(index[3*i:3*(i+1)])
    
print(np.ceil(1.1), np.ceil(1.7), np.ceil(2.7)) # ceilは切り上げ関数

index=[0 1 2 3 4 5 6 7 8 9]
index=[7 2 9 1 0 8 4 5 6 3]

[7 3 5 1 4 8 0 2 6 9]

[7 2 9]
[1 0 8]
[4 5 6]
[3]
2.0 2.0 3.0


In [20]:
np.random.seed(1234)
train_size = train_labels.shape[0]
batch_size = 32
epochs = 10
network = None #ダミー
minibatch_num = np.ceil(train_size/batch_size).astype(int) # ミニバッチの個数
    
for epoch in range(epochs):
    print()
    
    # indexを定義し、シャッフルする
    index = np.arange(train_size)
    np.random.shuffle(index)
    
    for mn in range(minibatch_num):
        """
        非復元抽出によるループ
        """
        batch_mask = index[batch_size * mn : batch_size * (mn + 1)]
        if np.mod(mn, 100)==0:
            print("epoch=%s, "%epoch,"mn=%s, "%mn , "batch_mask=%s"%batch_mask[:10])
        x_batch = train[batch_mask]
        y_batch = train_labels[batch_mask]

        trainer(network, x_batch, y_batch)


epoch=0,  mn=0,  batch_mask=[30329 44957 30866 40447 25580  6216 26373  9010 23445   108]
epoch=0,  mn=100,  batch_mask=[13196 36161 19297 47668 43204 53789 20787 28580 15708 28730]
epoch=0,  mn=200,  batch_mask=[57861 11320 48581 10823   122  3304 36073 51240 52366 26261]
epoch=0,  mn=300,  batch_mask=[30832 12049 14714  8507 14579 44748 46360 51998 31112 20184]
epoch=0,  mn=400,  batch_mask=[55463 58348 47584  5632 34691  5580 14161 31994 24594 51992]
epoch=0,  mn=500,  batch_mask=[13286 42771 30701 36253 20841 18060 26171 16097 53271  2561]
epoch=0,  mn=600,  batch_mask=[17394 29041 56149 55570 19332 40836 45961 14389 28184 35640]
epoch=0,  mn=700,  batch_mask=[29468  4704 57628 25574 47253 57210 48338 55438  9637  7583]
epoch=0,  mn=800,  batch_mask=[  541 14048  7522 39030 20557  5302 57769 56218 57342  9972]
epoch=0,  mn=900,  batch_mask=[36523 49833 41420 58800 50758 34552 55193 44471  3732 58420]
epoch=0,  mn=1000,  batch_mask=[32066  2545 23122 10582 54259 39464 34687 23687 2